<a href="https://colab.research.google.com/github/AxelGautrand/Digitalization-in-Sports/blob/main/statsbomb_runonce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

In [3]:
# Uncomment these lines (only once)
#%cd drive/My\ Drive/DIS/Clone
#!git clone https://github.com/statsbomb/open-data.git

In [4]:
!pip install statsbombpy
clear_output()

In [5]:
dataset = "/content/drive/My Drive/DIS/Colab/open-data/data/"

In [8]:
from statsbombpy import sb
import pandas as pd
import json
import os

In [7]:
!pwd
writePath = "drive/My Drive/DIS/Write"
%cd $writePath

/content
/content/drive/My Drive/DIS/Write


In [9]:
from statsbombpy import sb
import json
import os

class CreateEvent360():
    def __init__(self):
        self.competitionId = 53
        self.seasonId = 106
        self.fmt = "dict"
        self.matchInfo = self.get_all_matches_info()
        self.matchId = list(self.matchInfo.keys())
        self.matchEvents = dict(zip(self.matchId, list(map(self.get_all_events_from_match, self.matchId))))
        self.matchFrames = dict(zip(self.matchId, list(map(self.get_all_360_frames_from_match, self.matchId))))
        self.matchEventLocations = dict(zip(self.matchId, list(map(self.get_event_id_matching_360_events, self.matchId))))

        for iMatchId in self.matchId:
            self.fill_event_frame_location_details(iMatchId)
            self.exportJson(iMatchId)


    def get_all_matches_info(self):
        return sb.matches(competition_id=self.competitionId, season_id=self.seasonId, fmt=self.fmt)

    def get_all_events_from_match(self, matchId):
        return sb.events(matchId, fmt=self.fmt)

    def get_all_360_frames_from_match(self, matchId):
        return sb.frames(matchId, fmt=self.fmt)

    def get_event_id_matching_360_events(self, matchId):
        eventsIds = [self.matchFrames[matchId][iEventIndex]['event_uuid'] for iEventIndex in
                     range(len(self.matchFrames[matchId]))]
        #return dict(zip(eventsIds, len(eventsIds) * [{'possession': None, 'eventLoc': None, 'homeLoc': list(), 'awayLoc': list()}]))
        return {x: {'possession': None, 'eventLoc': None, 'homeLoc': list(), 'awayLoc': list()} for x in eventsIds}

    def fill_event_frame_location_details(self, matchId):
        for iEventIndex in range(len(self.matchFrames[matchId])):
            iEvent = self.matchFrames[matchId][iEventIndex]['event_uuid']
            if 'location' in self.matchEvents[matchId][iEvent].keys():
                self.matchEventLocations[matchId][iEvent]['eventLoc'] = self.matchEvents[matchId][iEvent]['location']
            possessionId = self.matchEvents[matchId][iEvent]['possession_team']['id']
            if possessionId == self.matchInfo[matchId]['home_team']['home_team_id']:
                possession = 'home'
            else:
                possession = 'away'
            self.matchEventLocations[matchId][iEvent]['possession'] = possession
            for iPlayer in self.matchFrames[matchId][iEventIndex]['freeze_frame']:
                if possession == 'home':
                    if iPlayer['teammate']:
                        self.matchEventLocations[matchId][iEvent]['homeLoc'].append(iPlayer['location'])
                    else:
                        self.matchEventLocations[matchId][iEvent]['awayLoc'].append(iPlayer['location'])
                elif possession == 'away':
                    if iPlayer['teammate']:
                        self.matchEventLocations[matchId][iEvent]['awayLoc'].append(iPlayer['location'])
                    else:
                        self.matchEventLocations[matchId][iEvent]['awayLoc'].append(iPlayer['location'])
                        
    def exportJson(self, matchId):
          fileName = str(matchId) + '.json'
          with open(fileName, 'w') as fh:
              json.dump(self.matchEventLocations[matchId], fh)
          fh.close()

In [10]:
event360 = CreateEvent360()

/usr/local/lib/python3.7/dist-packages/statsbombpy/api_client.py:20: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn("credentials were not supplied. open data access only", NoAuthWarning)


In [11]:
def loadJson(matchId):
    filename = str(matchId) + '.json'
    with open(filename, 'r') as fh:
        matchData = json.load(fh)
    fh.close()
    return matchData

In [12]:
matchdata = loadJson(3847567)